In [1]:
import cv2
import numpy as np
import pandas as pd
import os
import tqdm
# from concurrent.futures import ThreadPoolExecutor

In [2]:
df = pd.DataFrame(columns=['Video Name',
                           'Average Brightness',
                           'Average Contrast',
                           'Average Spatial Noise',
                           'Average Temporal Noise',
                           'Average Colorfulness Index',
                           'Average Color Banding',
                           'Average Blockiness',
                           'Average Sharpness',
                           'Average Blur'])

In [3]:
def calculate_video_brightness(video_path):
    """
    Calculates the average and median brightness of a video.

    Args:
    - video_path (str): Path to the video file.

    Returns:
    - avg_brightness (float): Average brightness across all frames of the video.
    - median_brightness (float): Median brightness across all frames of the video.
    """
    cap = cv2.VideoCapture(video_path)

    brightness_scores = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate mean brightness of the grayscale frame
        mean_brightness = np.mean(gray)
        brightness_scores.append(mean_brightness)

    cap.release()

    # Aggregate brightness scores across all frames
    avg_brightness = np.mean(brightness_scores) if brightness_scores else 0

    return avg_brightness

In [4]:
def calculate_frame_contrast(frame):
    """
    Calculates the RMS contrast for a single video frame.

    Args:
    - frame (ndarray): The video frame in BGR format.

    Returns:
    - rms_contrast (float): The RMS contrast of the frame.
    """
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate mean and standard deviation of pixel intensities
    mean_intensity = np.mean(gray)
    std_intensity = np.std(gray)

    # RMS contrast is the standard deviation of the pixel intensities
    rms_contrast = std_intensity / mean_intensity if mean_intensity != 0 else 0

    return rms_contrast

def calculate_video_contrast(video_path):
    """
    Calculates the average and median RMS contrast of a video.

    Args:
    - video_path (str): Path to the video file.

    Returns:
    - avg_contrast (float): Average RMS contrast across all frames of the video.
    - median_contrast (float): Median RMS contrast across all frames of the video.
    """
    cap = cv2.VideoCapture(video_path)

    contrast_scores = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Calculate contrast for the current frame
        frame_contrast = calculate_frame_contrast(frame)
        contrast_scores.append(frame_contrast)

    cap.release()

    # Aggregate contrast scores across all frames
    avg_contrast = np.mean(contrast_scores) if contrast_scores else 0

    return avg_contrast


In [5]:
def calculate_frame_noise(frame, block_size=8):
    """
    Estimates the noise level in a single frame using spatial noise estimation.

    Args:
    - frame (ndarray): The video frame in grayscale format.
    - block_size (int): Size of the local window to compute local statistics.

    Returns:
    - noise_level (float): Estimated noise level in the frame.
    """
    # Convert frame to grayscale if it is not already
    if len(frame.shape) == 3:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    else:
        gray = frame

    # Calculate local mean and standard deviation in blocks of size block_size x block_size
    mean_local = cv2.blur(gray, (block_size, block_size))
    sq_diff = (gray - mean_local) ** 2
    std_local = np.sqrt(cv2.blur(sq_diff, (block_size, block_size)))

    # Estimate noise level as the mean of the local standard deviations
    noise_level = np.mean(std_local)

    return noise_level

def calculate_video_noise(video_path, block_size=8):
    """
    Calculates the average noise level in a video by analyzing frame differences and spatial noise.

    Args:
    - video_path (str): Path to the video file.
    - block_size (int): Size of the local window to compute local statistics for spatial noise estimation.

    Returns:
    - avg_noise (float): Average noise level across all frames of the video.
    - temporal_noise (float): Temporal noise estimated from consecutive frame differences.
    """
    cap = cv2.VideoCapture(video_path)

    noise_scores = []
    temporal_diffs = []

    ret, prev_frame = cap.read()
    if not ret:
        cap.release()
        return 0, 0

    # Convert first frame to grayscale
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert current frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate spatial noise for the current frame
        frame_noise = calculate_frame_noise(gray, block_size)
        noise_scores.append(frame_noise)

        # Calculate temporal noise based on frame difference
        frame_diff = cv2.absdiff(gray, prev_gray)
        temporal_noise = np.std(frame_diff)
        temporal_diffs.append(temporal_noise)

        # Update previous frame
        prev_gray = gray

    cap.release()

    # Aggregate noise scores across all frames
    avg_noise = np.mean(noise_scores) if noise_scores else 0
    avg_temporal_noise = np.mean(temporal_diffs) if temporal_diffs else 0

    return avg_noise, avg_temporal_noise

In [6]:
def calculate_colorfulness(frame):
    # Convert frame to RGB
    (B, G, R) = cv2.split(frame.astype("float"))

    # Compute rg = R - G
    rg = np.abs(R - G)

    # Compute yb = 0.5 * (R + G) - B
    yb = np.abs(0.5 * (R + G) - B)

    # Compute the mean and standard deviation of both `rg` and `yb`
    (rb_mean, rb_std) = (np.mean(rg), np.std(rg))
    (yb_mean, yb_std) = (np.mean(yb), np.std(yb))

    # Combine the mean and standard deviations
    std_root = np.sqrt((rb_std ** 2) + (yb_std ** 2))
    mean_root = np.sqrt((rb_mean ** 2) + (yb_mean ** 2))

    # Compute the colorfulness metric
    colorfulness = std_root + (0.3 * mean_root)
    return colorfulness

def calculate_video_colorfulness(video_path):
    cap = cv2.VideoCapture(video_path)

    colorfulness_scores = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Calculate colorfulness for the current frame
        colorfulness = calculate_colorfulness(frame)
        colorfulness_scores.append(colorfulness)

    cap.release()

    # Average colorfulness across all frames
    avg_colorfulness = np.mean(colorfulness_scores)

    return avg_colorfulness

In [7]:
def calculate_frame_color_banding(frame):
    """
    Calculates the color banding score for a single video frame using chroma noise analysis.

    Args:
    - frame (ndarray): The video frame in BGR format.

    Returns:
    - color_banding_score (float): Color banding score of the frame.
    """
    # Convert frame to YUV color space
    yuv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2YUV)

    # Extract U and V channels
    U = yuv_frame[:, :, 1]
    V = yuv_frame[:, :, 2]

    # Calculate differences between adjacent pixels in U and V channels
    diff_U_vert = np.abs(np.diff(U, axis=0))  # Vertical differences
    diff_U_horiz = np.abs(np.diff(U, axis=1))  # Horizontal differences

    diff_V_vert = np.abs(np.diff(V, axis=0))  # Vertical differences
    diff_V_horiz = np.abs(np.diff(V, axis=1))  # Horizontal differences

    # Calculate mean of differences for each direction
    mean_diff_U = (np.mean(diff_U_vert) + np.mean(diff_U_horiz)) / 2
    mean_diff_V = (np.mean(diff_V_vert) + np.mean(diff_V_horiz)) / 2

    # Calculate color banding score as the average of the U and V differences
    color_banding_score = (mean_diff_U + mean_diff_V) / 2

    return color_banding_score

def calculate_video_color_banding(video_path):
    """
    Calculates the average color banding score of a video.

    Args:
    - video_path (str): Path to the video file.

    Returns:
    - avg_color_banding (float): Average color banding score across all frames of the video.
    """
    cap = cv2.VideoCapture(video_path)

    color_banding_scores = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Calculate color banding score for the current frame
        frame_color_banding = calculate_frame_color_banding(frame)
        color_banding_scores.append(frame_color_banding)

    cap.release()

    # Aggregate color banding scores across all frames
    avg_color_banding = np.mean(color_banding_scores) if color_banding_scores else 0

    return avg_color_banding

#### Pixel Intensity Difference Technique

The original method calculates the blockiness score by analyzing the intensity differences across block boundaries in a grayscale frame:
- Horizontal Blockiness: It computes the absolute differences in pixel intensities between adjacent horizontal rows at every block boundary (every 8 pixels in height).
- Vertical Blockiness: Similarly, it computes the absolute differences between adjacent vertical columns at every block boundary (every 8 pixels in width).
- The blockiness score is then obtained by summing these differences and normalizing them by the total number of boundaries and pixels.

In [8]:
def calculate_frame_blockiness(frame):
    """
    Calculates the blockiness score for a single video frame using DCT coefficients.

    Args:
    - frame (ndarray): The video frame in BGR format.

    Returns:
    - blockiness_score (float): Blockiness score of the frame.
    """
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Parameters for block size
    block_size = 8
    height, width = gray.shape

    blockiness_horizontal = 0
    blockiness_vertical = 0

    # Calculate horizontal blockiness (difference across horizontal block boundaries)
    for i in range(block_size, height, block_size):
        blockiness_horizontal += np.sum(np.abs(gray[i, :] - gray[i-1, :]))

    # Calculate vertical blockiness (difference across vertical block boundaries)
    for j in range(block_size, width, block_size):
        blockiness_vertical += np.sum(np.abs(gray[:, j] - gray[:, j-1]))

    # Normalize blockiness score by the number of boundaries and pixels
    total_blocks = (height // block_size - 1) * width + (width // block_size - 1) * height
    blockiness_score = (blockiness_horizontal + blockiness_vertical) / total_blocks if total_blocks > 0 else 0

    return blockiness_score

def calculate_video_blockiness(video_path):
    """
    Calculates the average blockiness score of a video.

    Args:
    - video_path (str): Path to the video file.

    Returns:
    - avg_blockiness (float): Average blockiness score across all frames of the video.
    """
    cap = cv2.VideoCapture(video_path)

    blockiness_scores = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Calculate blockiness score for the current frame
        frame_blockiness = calculate_frame_blockiness(frame)
        blockiness_scores.append(frame_blockiness)

    cap.release()

    # Aggregate blockiness scores across all frames
    avg_blockiness = np.mean(blockiness_scores) if blockiness_scores else 0

    return avg_blockiness

In [9]:
def calculate_frame_sharpness(frame):
    """
    Calculates the sharpness score for a single video frame using the variance of the Laplacian.

    Args:
    - frame (ndarray): The video frame in BGR format.

    Returns:
    - sharpness_score (float): Sharpness score of the frame.
    """
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Compute the Laplacian of the image and then the variance
    laplacian = cv2.Laplacian(gray, cv2.CV_64F)
    sharpness_score = laplacian.var()

    return sharpness_score

def calculate_frame_blur(frame):
    """
    Calculates the blur score for a single video frame using the Sobel operator.

    Args:
    - frame (ndarray): The video frame in BGR format.

    Returns:
    - blur_score (float): Blur score of the frame.
    """
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Compute Sobel gradients in x and y directions
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=5)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=5)

    # Compute gradient magnitude
    gradient_magnitude = np.sqrt(sobelx**2 + sobely**2)

    # Blur score is the inverse of the variance of gradient magnitudes
    blur_score = 1 / (np.var(gradient_magnitude) + 1e-6)  # Adding a small epsilon to avoid division by zero

    return blur_score

def calculate_video_quality_metrics(video_path):
    """
    Calculates the average sharpness and blur scores of a video.

    Args:
    - video_path (str): Path to the video file.

    Returns:
    - avg_sharpness (float): Average sharpness score across all frames of the video.
    - avg_blur (float): Average blur score across all frames of the video.
    """
    cap = cv2.VideoCapture(video_path)

    sharpness_scores = []
    blur_scores = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Calculate sharpness and blur scores for the current frame
        frame_sharpness = calculate_frame_sharpness(frame)
        frame_blur = calculate_frame_blur(frame)

        sharpness_scores.append(frame_sharpness)
        blur_scores.append(frame_blur)

    cap.release()

    # Aggregate sharpness and blur scores across all frames
    avg_sharpness = np.mean(sharpness_scores) if sharpness_scores else 0
    avg_blur = np.mean(blur_scores) if blur_scores else 0

    return avg_sharpness, avg_blur

In [ ]:
original_folder = "Code\\Epic_vid_balanced_new_7Sept"

all_videos = [os.path.join(original_folder, video) for video in os.listdir(original_folder) if video.endswith('.mp4')]

In [11]:
for video in tqdm.tqdm(all_videos, desc="Processing Videos", unit="video"):

  video_name = os.path.basename(video).split('\\')[-1]
  
  avg_brightness = calculate_video_brightness(video)
  avg_contrast = calculate_video_contrast(video)
  spatial_noise, temporal_noise = calculate_video_noise(video, block_size=8)
  avg_colorfulness_index = calculate_video_colorfulness(video)
  avg_color_banding = calculate_video_color_banding(video)
  avg_blockiness = calculate_video_blockiness(video)
  avg_sharpness, avg_blur = calculate_video_quality_metrics(video)

  new_row = {'Video Name' : video_name,
             'Average Brightness' : avg_brightness,
             'Average Contrast' : avg_contrast,
             'Average Spatial Noise': spatial_noise,
             'Average Temporal Noise': temporal_noise,
             'Average Colorfulness Index': avg_colorfulness_index,
             'Average Color Banding': avg_color_banding,
             'Average Blockiness': avg_blockiness,
             'Average Sharpness': avg_sharpness,
            'Average Blur': avg_blur}

  df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

Processing Videos:   0%|          | 0/16800 [00:00<?, ?video/s]C:\Users\maury\AppData\Local\Temp\ipykernel_21008\3380749901.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
Processing Videos: 100%|██████████| 16800/16800 [4:59:32<00:00,  1.07s/video]   


In [12]:
df

,Video Name,Average Brightness,Average Contrast,Average Spatial Noise,Average Temporal Noise,Average Colorfulness Index,Average Color Banding,Average Blockiness,Average Sharpness,Average Blur
0,P01_01_1.mp4,135.500049,0.340637,0.612793,8.456470,19.497973,8.575249,73.569635,2.683567,6.286984e-05
1,P01_01_102.mp4,162.289770,0.354829,3.019531,23.252530,13.805552,14.329576,98.647440,107.111174,2.288238e-06
2,P01_01_103.mp4,99.984907,0.330643,0.993164,9.638859,17.467424,14.717883,87.071033,4.259571,6.431504e-05
3,P01_01_109.mp4,186.054577,0.316723,3.388672,28.927842,26.927166,24.063380,96.172040,219.579275,2.481337e-06
4,P01_01_115.mp4,178.583482,0.375943,3.691406,35.235893,19.091039,16.176031,88.101402,432.249234,1.064713e-06
...,...,...,...,...,...,...,...,...,...,...
16795,P37_103_59.mp4,84.305491,0.413508,1.227539,8.788208,34.904741,23.340270,87.417189,5.198416,5.506206e-05
16796,P37_103_63.mp4,96.443705,0.375974,0.797852,16.946744,26.635863,11.115104,77.431260,3.599347,1.289816e-04
16797,P37_103_71.mp4,76.634427,0.619754,2.476562,17.116973,40.338540,26.503030,94.999621,9.747126,8.410487e-06
16798,P37_103_73.mp4,58.953690,0.592401,3.988281,10.067540,41.346164,28.262763,102.351023,144.834311,3.132583e-06


In [13]:
df.to_csv("features.csv")

#### Merging the Scores of the videos to the features of the videos so that they can be fed into the MLP Model.

In [ ]:
df_scores = pd.read_csv("Code\\NEW\\balanced_dataset_details_7thSept.csv")